## 量化和动态量化 and 量化感知

量化：将"占用较多比特的浮点数"（高比特数）转化成 "占用较少比特的浮点数or整数"（低比特数）。核心目的是 减少模型大小、加快数据传输、提高计算效率。  
  
由于 低比特数 天然使用更少 比特位，信息量少，所以低比特数最多能表示的数也是偏少的。比如 高比特数 X range范围 [-5.0, 5.0]，如果用低比特数 INT8 去表示，INT8 是 [-128, 127]，  
那么能拿来表示 X 的就只有 -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5 这十一个数，精度过于低了。所以要采用（线性）缩放的方式，把 高比特数 的 range 范围映射到 低比特数的  
整个 range 范围，以充分利用 低比特数的 range 范围。  
  
缩放因子 S: 高比特数的 range 范围 [min, max]，低比特数 的 range 范围 [Q_min, Q_max]   --->   S = (max - min)/(Q_max - Q_min)  
缩放因子应该是一个 很小的 数值，因为只有 小（小于1），把 高比特的 窄range 映射到 低比特的 宽range，才能保持信息尽量不丢失。窄range/宽range自然就得到了一个很小的数值  
  
除了缩放，可能还会有偏移。即 max 应该 映射到 Q_max, min 应该映射到 Q_min, 这个偏移量 Z = Q_max - max/S = Q_min - min/S  （恒等式，从S的定义可推导）  
这样 max/S + Z = Q_max, min/S + Z = Q_min, 满足条件。可以看出这个 Z 的意义是 高比特数的 0 映射到 低比特数 Z  
  
综上，量化的过程是：  
    高比特数 X 的 range 范围 [min, max], Quantization 低比特数 的range 范围 [Q_min, Q_max],   
    定义缩放因子 scale factor S 为 "高比特range宽度 除以 低比特range宽度":= (max - min)/(Q_max - Q_min)  
  
    为了保证映射"两端对齐", 得出一个 低比特数范围内的偏移量 zero-point Z := Q_max - max/S or Q_min - min/S  
    由此得到量化的映射公示:  
        对于 range 范围 [min, max] 的高比特数 X, 量化到 range 范围 [Q_min, Q_max] 的低比特数 Y, 则有:  
        S = (max-min)/(Q_max-Q_min), Z = Q_max - max/S or Q_min - min/S  
          
        Y rounding:=  X/S + Z  
  
对称量化:  
    高比特数 X 只求一个 绝对值最大值 Amax, 将其 range 范围设定为 [-Amax, Amax]，关于 0 对称。  
    绝大多数情况下，低比特数 是带符号的，那么其 range 也会是关于 0 对称，因为最小负数等于-最大正数，[-Q_max, Q_max]  
    这样缩放因子 S = Amax/Q_max, Z = Q_max - Amax/S = 0  
  
    Y rounding:= X/S, S = Amax/Q_max  
  
  
总结，缩放因子的使用: 高比特数 除以 S --rounding--> 低比特数，低比特数 乘以 S --exact--> 高比特数  
     偏移 Z 发生在 低比特数，高比特数/S + Z --> 对应低比特数，（低比特数-Z）* S --> 对应高比特数。如果采用对称量化+带符号的低比特数，那么 偏移 Z永远为 0  
  


从上述量化的过程可以看出，量化前的 高比特数 X，和量化后对应的 低比特数 Q_X，在数值上是线性映射的关系。  
  
量化是为了把高比特数之间的运算，放在低比特数之间以加速运行。但归根结底是为了正确的结果，而不是为了加速而得到一个错误的答案。所以低比特数的运算结果，应该保证可以用  
反量化的方式映射回高比特数，且这个高比特数跟“用高比特数直接运算的结果”要近似。即：  
    变量 X1 -量化-> Q_X1，X2 -量化-> Q_X2  
    Y = OP( X1, X2 ), Q_Y = OP( Q_X1, Q_X2 ), 需要保证 Q_Y -反量化-> Y  
    量化和反量化都是线性操作。可以证明，OP运算自身也必须是线性运算，才能使得结果Y可以被线性反量化  
  
如果 OP运算 涉及 非线性运算，怎么办？  
1. 在非线性运算之前，插入反量化操作，即 对线性运算结果 作反量化，用高比特数参与非线性运算。非线性运算的结果可以重新通过量化，进入后续操作。  
2. 把非线性运算 替换为 近似的线性运算  
    比如 RELU：作用在高比特数上，大于0的保留，小于等于0的赋值为0，替换成 近似的Q_RELU：作用在低比特数上，大于Z的保留，小于等于Z的赋值为Z。  
    再比如 tanh，替换成分段线性Q_tanh，或者干脆用查表（Look Up Table），构建 X_低比特 至 Y_高比特（甚至直接到Y_低比特） 的映射。  
3. 消除非线性操作。对于有些操作，比如 Batch Normalize，在静态量化中通常会将其操作融合到其前面的线性层（如卷积层或全连接层）的权重和偏置中。  
    这样，BN 层在推理时就不再作为一个单独的层存在，从而消除了 BN 层的量化问题。  
  
方法2和方法3在实际运算中，尽量避免了频繁的量化、反量化开销。不过在计算图分析的时候，还是应该把量化和反量化节点（Q和DQ节点）插入到层的前、后，表示这个层是量化版本：  
    即：在模型图中，通常会在层输入之前插入一个量化操作，在层输出之后插入一个反量化操作（或在下一层的输入处量化）。  
这个反量化操作是必要的，因为它将整数结果转换回浮点数，以便与未量化的部分或需要浮点输入的层兼容，并进行后续的层间操作。    
如果使用方法2和方法3使用得当，反量化操作经常是"概念性"的，因为实际计算会尽可能保持在整数域。  
  
根治非线性层量化不适用的办法：QAT训练  


### 静态量化 是一种 后训练优化技术：after训练、浮点权重已确定

静态量化不涉及梯度和梯度状态，主要作用在 层输入X、输出Y、层权重W、层偏置B。  
上文已经说明了，量化和反量化操作 不影响运算结果的正确性的前提，是"运算必须是线性运算"。非线性运算可以用提前反量化、线性近似、LUT等方法解决。在深度学习中，最主要的
线性运算就是 Y = X @ W + B，其中 X 是层输入或初始输入节点，W 是层权重，B 是层偏置，Y是层输出，且回事下一层的层输入。它们在静态量化中经历的具体操作完全不同。  
  
下文 用大写字母表示某变量的 高比特数版本，小写字母表示 低比特数版本，@ +. 分别代表高比特数版本的乘法、加法，* + 分别代表低比特数版本的乘法、加法，  
  
Y <= X @ W +. B      希望用低比特版本         y <= x * w + b        完成运算，然后 y --反量化--> Y 用以其他运算，或者干脆保持低比特数 y 参与后续运算  

1. 层权重 W 的静态量化：如何找到 W 的低比特版本数 w？  
层权重 W，在模型 load 的时候，就完成静态量化。此时高比特数 W 都已经全部确定，可以找到它的 min/max 或 Amax，以及（带符号或不带符号的）目标规范（INT8、FP8、FP16等），
就能确定 Sw 和 Zw，继而 w := W/Sw + Zw

2. 层输入 X 的静态量化：如何找到 X 的低比特版本数 x?
层输入 X，首先有一个"校准" calibration 阶段：取一个具有代表性的真实数据集 C，以高比特数跑一遍高比特版模型，得到 X 和 Y 的代表性数据。以此得到 X 的 min/max 或 Amax，
确定 Sx 和 Zx。  
在真实推理阶段，真实的 高比特数层输入 X 来了，有 x := X/Sx + Zx

3. 层偏置 B 和 层输出 Y 的静态量化
如何找到 B 的低比特版本数 b?
尽管层偏置 B 在模型load时就确定了，但它的缩放因子 Sb，是在"校准"之后才确定的，且Zb不需要去确定。  
原因是为了 保持 y = x * w + b 仍然是 Y = X @ W +. B 的线性关系。具体推导是这样的：  
根据量化映射，有   
    低比特数 y = (X/Sx + Zx) * (W/Sw + Zw) + (B/Sb + Zb)，这里 Sb 和 Zb 待定，因为要分析它们和其他参数之间的关系。Sw Zw Sx Zx 都已经确定  
拆开式子，有   
    y = (X * W)/(Sx*Sw) + B/Sb + offset + Zb，这里 offset = X/Sx*Zw + W/Sw*Zx + Zx*Zw
  
考虑 Y 的量化 y = Y/Sy + Zy，以及高比特 Y = X @ W + B 代入，y = (X @ W)/Sy + B/Sy + Zy   
对比 y 的两个表达式子可以看出：  
    1. B的系数 应该和 X与W的乘积的系数 相同，这样才能保持 Y 到 y 的映射是线性的。故 Sb = Sx * Sw  
    2. Sb 应该等于 Sy 应该等于 Sx * Sw, Zy 应该等于 offset + Zb  
  
首先，应该有 Sb = Sy = Sx * Sw。考虑到 缩放因子是很小的数值，说明 缩放因子 Sb 和 Sy 在数值上远小于 Sx 和 Sw，  
说明 B 和 Y 的量化（至少Y的第一时间运算结果 y' 的量化）用低比特数是不够存储信息的，必须要用比 X 和 W 更多比特才行（缩放因子分母的比特数越多，range越宽，缩放因子越小）。  
从第一性原理考虑，也确实是这样。假如 X 和 W 都用 INT8 量化，一个20，一个40，那么此时的运算结果 800，y 如果是INT8量化，根本就存不下。  
故：B 和 Y 的量化（至少Y的第一时间运算结果 y' 的量化），应该采用 中比特位数（比x和w要多）。如果最终仍然需要 Y 的低比特数版本，应该再作一次 中比特 y' 至 低比特 y 的量化映射。  
  
关于层偏置 B，上文已经说明了 b 应该是 B 的 中比特版本，即 b 的比特数应该多于 x 和 w，尽管 b 相对 B 还是 低比特的。  
缩放因子 Sb := Sx * Sw ， Zb 从后面可以发现，不需要计算了，故不再谈。  
  
  
将 Sb = Sx * Sw 代入得到  
    y = (X/Sx) * (W/Sw) + B/Sb + offset + Zb = (X * W + B)/(Sx*Sw) + offset + Zb = Y/(Sx*Sw) + offset + Zb  
显然等式右边是 中比特数的 运算，故 这里等式左边 y 是 中比特数 的运算结果，用 y' 重新标记下，得到  
    y' = Y/Sy' + Zy', 这里 Sy' = Sx*Sw ，Zy' = offset + Zb  
  
重新阐述一下，Sy 和 Zy 是在"校准"阶段获得，上述又说明 Sy 和 Zy 可以通过Sx、Sw、Zx、Zw、W、X等计算出来，似乎有多重方法可得到。选哪个来源？  
实际上，"校准"的，和算出来的，是两个不同尺寸下的缩放因子和零点。  
Sy 和 Zy 是在"校准"阶段获得，由于"校准"阶段是设定了 Y 的目标量化版本（低比特）的，故 Sy 和 Zy 是 Y 映射到 低比特数 y 的缩放因子和零点  
而上述计算出来的 Sy' = Sx * Sw , Zy' = offet + Zb，因为是在等式右边 中比特 range范围内算出来的，故它们其实是 Y 映射到 中比特数 y' 的缩放因子和零点  
  
  
  
  
如何得到 Y 的低比特版本数 y? Y 的 中比特版本数 y'  
有公式 y' = x * w + b。但实际上这个式子并不好用。分析一下，这里 x 已知、w 已知，但是 b = B/Sb + Zb，尽管 Sb = Sx * Sw 已知，但是 Zb 并没有去计算。  
换一个式子：y' = (X * W)/(Sx*Sw) + B/Sb + offset + Zb = (X/Sx) * (W/Sw) + B/(Sx*Sw) + Zy'，即：  
    y' - Zy' = (x - Zx)*(w - Zw) + B/(Sx*Sw)  
  
这里 X/Sx = x - Zx ，W/Sw = w - Zw 都是低比特(比如int8)数。而它们的乘积是累加得到的（低比特运算加速），累加结果 应该用 中比特储存。  
B 的缩放因子是 Sx*Sw，上文分析过了，也应该用一个 中比特数 去存储 中比特数 B/(Sx*Sw)。所以等式右边 的结果记为 中比特数 Acc，隐藏的缩放因子 Sx * Sw，即：  
    y' - Zy' = Acc              这里 Acc 是 低比特(x - Zx) 乘以 低比特(w - Zw) 加 中比特数 B/(Sx*Sw)，存储在 中比特 Acc 的结果  
  
同时，还有 Y 到 中比特版本 y'的映射，上文已经论述过，缩放因子 Sy' = Sx * Sw  
    y - Zy' = Y/Sy' = Y/(Sx * Sw)  
  
从而立刻得到 Acc 就是 Y/(Sx * Sw) 。 从而得到 Y 的 高比特数 Y = Acc * Sx * Sw 。从而直接得到 低比特版本数 y 有：  
    y = Y/Sy + Zy = (Acc * Sx * Sw)/Sy + Zy = Acc * (Sx * Sw)/Sy + Zy 。把 Acc 重新用 低->中比特数运算 代入，得到  
  
    y = [ (x - Zx)*(w - Zw) + B/(Sx*Sw) ] * (Sx * Sw)/Sy + Zy  
  
    这里，x 是 输入 X 的 低比特量化版本，缩放因子和零点分别为 Sx 和 Zx。缩放因子/零点 是 校准阶段确定，x 是 X 输入时确定  
         w 是 权重 W 的 低比特量化版本，缩放因子和零点分别为 Sw 和 Zw。缩放因子/零点 是 模型加载确定，w 也是模型加载时就确定  
         b 是 偏置 B 的 中比特量化版本，缩放因子为 Sx*Sw，零点不需要考虑。缩放因子 是 校准阶段确定，b 也是校准阶段确定  
         Sy 和 Zy 是 输出 Y 的 低比特量化 的缩放因子和零点。校准阶段确定。同时确定 Multiplier = (Sx*Sw)/Sy  
           
         step 0: 模型加载、校准，确定 Sx, Zx, Sw, Zw, w, B/(Sx*Sw), Sy, Zy。   x 尚未确定，求 y  
         step 1:  
            输入 X，得到 低比特数 x   
         step 2:  
            [ (x - Zx)*(w - Zw) + B/(Sx*Sw) ] --->  Acc 是 中比特数累加结果  
         step 3:  
            Acc * Multiplier + Zy，这个浮点乘法用定点乘法近似加速  
         step 4:  
            rounding 上一步结果，并在 低比特range范围内 clipping  ---> y  
           
  

### 量化感知 QAT 是一种 训练策略：在训练中模拟量化

为了在训练阶段就考虑量化的影响，使模型对量化误差具有鲁棒性，业界发展出了 量化感知训练 (Quantization-Aware Training, QAT)。  
  
QAT 的核心思想是：  
  
在训练过程中模拟量化效应： QAT 不是直接将浮点数转换为整数并进行整数运算，而是在浮点运算的训练流程中，插入“伪量化”或“假量化”节点。  
伪量化 (Fake Quantization) 的机制：  
每个张量（包括权重和激活）首先被量化到低比特整数范围。  
然后，立即将这些整数值反量化回浮点数（例如 FP32）。  
模型在训练时，所有的计算仍然是浮点运算。   
通过这种“量化再反量化”的模拟过程，梯度可以在训练过程中“感知”到由于量化而引入的误差。这使得模型能够调整其权重，以最小化这些误差，从而在最终被真正量化为整数时，保持更高的精度。  
量化参数的更新： QAT 通常也会在训练过程中动态地更新量化参数（如缩放因子和零点），或者使用一些更高级的统计方法来确定它们，以更好地适应训练过程中数值范围的变化。  
优点： QAT 训练出的模型在量化后，通常能比静态量化达到更高的精度，有时甚至能接近原始的 FP32 模型。  
缺点： 训练过程更复杂，需要修改训练代码，并且训练时间可能会略微增加。  

量化感知训练 (Quantization-Aware Training, QAT) 的作用：  
  
QAT 是处理非线性层的最有效方法。  
在 QAT 训练过程中，虽然实际计算是 FP32，但会在非线性层的前后插入“伪量化”和“伪反量化”节点。（前面插入"伪量化"和“伪反量化”节点，后面也插入"伪量化"和“伪反量化”节点）  
这使得模型在训练时能够“感知”到非线性层两侧的量化误差。模型会学习如何调整其权重，以使非线性函数在量化输入和输出的情况下，其行为尽可能接近原始浮点模型。  
QAT 能够帮助模型在非线性层上更好地保持精度，即使在部署时使用 QDQ 策略或 LUT。  